### 좋아요수 / 댓글 분산 적용 기반 추천(명곡?)
- Input(title, artist) : contains로 찾기 => 결과 songId 한 개만 반환하도록 한다.
- I am 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 -> 가사분석해서 비슷한 노래 찾기 top5
- I am 찾기 -> 가사분석해서 비슷한 노래 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 top5
- 반환된 songId 들의 정보 출력

In [1]:
import numpy as np
import pandas as pd

In [32]:
df = pd.read_csv('data/melon_song.csv')
df.tail(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
6245,34183979,인생 한잔,정은지,포크/블루스 국내드라마,술꾼도시여자들 OST Part 2,빨간양말,INAN,20211029.0,https://cdnimg.melon.co.kr/cm2/album/images/10...,12,4512,사랑도 이별도 오늘은 잠시 미뤄보자\r\n불안한 청춘도 내일로 잠시 미뤄두자\r\n...,506225967 434154554
6246,31191637,6 o'clock,볼빨간사춘기,인디음악 포크/블루스,Red Diary 'Hidden Track',안지영,안지영,20180717.0,https://cdnimg.melon.co.kr/cm/album/images/101...,87,15668,It's 6 o'clock\r\n너와 나는 시간이 빨라서\r\n시간이 달라서\r\n...,458080033 442957041


In [37]:
# 데이터 처리를 위한 작업
df.date.fillna(0, inplace=True)
df['date'] = df.date.astype(int).astype(str)
df.fillna('', inplace=True)
df['comment_like_total'] = df.comment + df.like
df['songId'] = df.songId.astype(str)

In [38]:
# 한글 불용어 처리
with open('data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

In [39]:
from konlpy.tag import Okt
okt = Okt()

In [40]:
lyrics = []
for lyric in df.lyric:
    lyric = lyric.replace('\n', ' ')
    morphs = okt.morphs(lyric, stem=True)
    tmp = [word for word in morphs if word not in stop_words]
    lyrics.append(' '.join(tmp))
df['morphs'] = lyrics

In [41]:
# 학습할 데이터열 생성
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 3

In [42]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)

In [43]:

from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)

In [10]:
# 장르가 힙합이나 댄스 : 결과 안 좋거나 거의 비슷하다. 
# from sklearn.feature_extraction.text import CountVectorizer
# cvect = CountVectorizer(stop_words='english')
# total_cv = cvect.fit_transform(df.total)

In [44]:
indices = pd.Series(df.index, index=df.songId)

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [51]:
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim = cosine_similarity(total_cv)

In [52]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(31).tail(30).index
    return df.songId.iloc[song_indices]

In [60]:
# 키워드 바꿔가면서 찾아보기
keyword_title = 'Doc와 춤을'.lower()
keyword_artist = 'DJ DOC'.lower()
print(keyword_title, keyword_artist)

songIds = df[df.title.str.lower().str.contains(keyword_title) & df.artist.str.lower().str.contains(keyword_artist)]
songIds

doc와 춤을 dj doc


,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq,comment_like_total,morphs,total
2425,9390,Doc와 춤을,DJ DOC,댄스,Season‘s Greeting,김창렬,박해운,20001212,https://cdnimg.melon.co.kr/cm/album/images/000...,44,14328,젓가락질 잘해야만 밥을 먹나요\r\n잘못해도 서툴러도 밥 잘 먹어요\r\n그러나 주...,442706386 418666508,14372,젓가락 질 잘해야 만 밥 먹다 \r 잘못 하다 서툴다 밥 자다 먹다 \r 그러나...,젓가락 질 잘해야 만 밥 먹다 \r 잘못 하다 서툴다 밥 자다 먹다 \r 그러나...


In [54]:
find_songid = songIds.songId.values[0]
a = get_recommendation(find_songid, cosine_sim).to_frame()
a.head()

,songId
3661,35694868
3006,2544949
240,489091
3796,3864961
4594,70602


In [55]:

# df[df['songId'].isin(a.songId[1:6])].to_dict('records')
df[df['songId'].isin(a.songId[1:6])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]

,songId,title,artist,genre,album,lyricist,composer
240,489091,원더 우먼,조원선,국내영화,봄날의 곰을 좋아하세요? OST,윤종신,
2394,70696,취중진담(醉中眞談),전람회,발라드,Exhibition 2,김동률,김동률
3006,2544949,못해 (Feat. 美),포맨 (4MEN),발라드,The 3rd Generation (Special Album),민연재,윤민수(바이브)
3796,3864961,가수가 된 이유,신용재 (2F),발라드,24,민연재,임세준
4594,70602,천생연분,솔리드,R&B/Soul,Light Camera Action!,김수현,정재윤


In [56]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
sim_scores = pd.Series(cosine_sim_total[indices[find_songid]])
sim_scores.sort_values(ascending=False).head(11).tail(10)

3661    0.162954
3006    0.148939
5528    0.136449
834     0.136067
955     0.125582
3555    0.125297
5439    0.121876
6063    0.121769
3044    0.119704
3718    0.119437
dtype: float64

In [30]:
# 찾고 싶은 구간 정하기
numbers = df['comment_like_total']
sorted_numbers = np.sort(numbers)
q1 = np.percentile(sorted_numbers, 25)
q2 = np.percentile(sorted_numbers, 50)
filtered_data = df[(df['comment_like_total'] >= q1) & (df['comment_like_total'] < q2)]
filtered_data = filtered_data[['songId', 'comment_like_total']]
filtered_data.songId.values

array(['35837631', '35866717', '35879546', ..., '3080890', '2981597',
       '34183979'], dtype=object)

In [19]:
# 유사도 top 30 중 원하는 구간에 있는 songId 추출(유사도순)
d = a[a['songId'].isin(filtered_data.songId.values)].head(5)
d

,songId
5377,36263930
4835,36421590
6101,31037943
4776,36282049
477,36425393


In [20]:
# filtered_df = df[df['songId'].isin(a.songId.values[:5])].to_dict('records')
filtered_df = df[df['songId'].isin(a.songId.values[:5])][['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer']]
filtered_df

,songId,title,artist,genre,album,lyricist,composer
3102,3837683,Loving U (러빙유),씨스타,댄스,Summer Special 'Loving U',이단옆차기,이단옆차기
4033,36182838,ROAR,더보이즈 (THE BOYZ),댄스,THE BOYZ 8th MINI ALBUM [BE AWAKE],KENZIE,DEEZ
4835,36421590,이별클럽 (Feat. 이찬혁),Colde (콜드),R&B/Soul,Love Part 2,Colde (콜드),이찬혁
5377,36263930,기다릴게,PLAVE,록/메탈,ASTERUM,PLAVE,EL CAPITXN
6101,31037943,그런 연애 (Feat. 아인 of 모모랜드),그_냥,인디음악 포크/블루스,그런 연애 (Feat. 아인 of 모모랜드),그_냥,그_냥
